In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
import csv
from xlsxwriter.workbook import Workbook
import win32com.client
import PyPDF2
from shutil import copyfile
import zipfile
import shutil

In [3]:
pasta_calypso = r'\\drioc439pfs\PROD_FILES\DASH\OPERATIONS\Broker Dealer Offshore\Position (NY)\Equity\Calypso'
pasta_final = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore\Ajustes'
pasta_downloads = r'C:\Users\correigu\Downloads'

In [4]:
# Transforma de .csv para .xslx
def csv_to_excel(csvfile):
    workbook = Workbook(csvfile[:-4] + '.xlsx')
    worksheet = workbook.add_worksheet()
    with open(csvfile, 'rt', encoding='latin-1') as f:
        reader = csv.reader(f)
        for r, row in enumerate(reader):
            for c, col in enumerate(row):
                worksheet.write(r,c,col)
    workbook.close()

In [5]:
today = datetime.datetime.today().strftime('%Y%m%d')

## Calypso (pasta)

In [9]:
for file in os.listdir(pasta_calypso):
    if (today in file)&('Equity' in file):
        copyfile(pasta_calypso+'\\'+file, pasta_final+'\\'+file)

In [18]:
# Aplica a transformação para os files da pasta
for file in os.listdir(pasta_final):
    #print(file)
    csv_to_excel(pasta_final+'\\'+file)

In [19]:
for file in os.listdir(pasta_final):
    if '.csv' in file:
        os.remove(pasta_final+'\\'+file)

In [20]:
# Ajusta o nome do file
for file in os.listdir(pasta_final):
    if 'BD+WM' in file:
        base = 'CUserssvc_calypso_prod.DOMAINCalypsoEquity  Posição Custo Aluguel Clients BD+WM - '
    elif 'INT C' in file:
        base = 'CUserssvc_calypso_prod.DOMAINCalypsoEquity - Posição Custo Aluguel INT C - '
    elif 'PROP+ASSET' in file:
        base = 'CUserssvc_calypso_prod.DOMAINCalypsoEquity  Posição Custo Aluguel PROP+ASSET - '
    else:
        print (file)
    
    os.rename(pasta_final+'\\'+file, pasta_final+'\\'+base+file[-13:-5]+'.xlsx')

In [21]:
for file in os.listdir(pasta_final):
    try: 
        if 'BD+WM' in file:
            df = pd.read_excel(pasta_final+'\\'+file)
            data = df.columns[31]
        else:
            df = pd.read_excel(pasta_final+'\\'+file)
            data = df.columns[7]
        data = pd.to_datetime(data)
        ano = data.year
        mes = "{:02}".format(data.month)
        dia = "{:02}".format(data.day)

        string = str(ano)+str(mes)+str(dia)
        os.rename(pasta_final+'\\'+file, pasta_final+'\\'+file[:-13]+string+'.xlsx')
    except FileExistsError:
        print(file)    

In [22]:
for file in os.listdir(pasta_final):
    os.rename(pasta_final+'\\'+file, pasta_final[:-8]+'\\'+file)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\correigu\\OneDrive - Banco BTG Pactual S.A\\Desktop\\aqruivos_offshore\\Ajustes\\CUserssvc_calypso_prod.DOMAINCalypsoEquity - Posição Custo Aluguel INT C - 20181031.xlsx' -> 'C:\\Users\\correigu\\OneDrive - Banco BTG Pactual S.A\\Desktop\\aqruivos_offshore\\CUserssvc_calypso_prod.DOMAINCalypsoEquity - Posição Custo Aluguel INT C - 20181031.xlsx'

## Morgan

In [8]:
for file in os.listdir(pasta_final):
    morgan = pd.read_csv(pasta_final+'\\'+file, skipfooter=1, engine='python')
    data = pd.to_datetime(morgan['Value Date'].drop_duplicates().reset_index(drop=True)[0])
    ano = data.year
    mes = "{:02}".format(data.month)
    dia = "{:02}".format(data.day)
    string = str(ano)+str(mes)+str(dia)
    
    
    if '038CDB895' in file:
        base = '038CDB895_IN150DX - '
    elif '038CDBD08' in file:
        base = '038CDBD08_IN150DX - '
    
    os.rename(pasta_final+'\\'+file, pasta_final+'\\'+base+string+'.csv')
    os.rename(pasta_final+'\\'+base+string+'.csv', pasta_final[:-8]+'\\'+base+string+'.csv')

## UBS (e-mail)

In [15]:
email_sender = 'guilherme.correia@btgpactual.com'

In [16]:
outlook = win32com.client.Dispatch('outlook.application')
namespace = outlook.GetNamespace("MAPI")
dict_index = {}
i=1
for folder in namespace.Folders:
    dict_index.update({str(folder).lower():i})
    i+=1
root_folder = namespace.Folders.Item(dict_index[email_sender])
print(root_folder)

Guilherme.Correia@btgpactual.com


In [17]:
subfolder = root_folder.Folders['Caixa de Entrada'].Folders['Aluguel offshore']

In [18]:
message = subfolder.Items

In [13]:
for msg in message:
    msg_sub = msg.Subject
    if 'Multiclient' in msg_sub:
        count_attachments = msg.Attachments.Count
        if count_attachments > 0:
            for item in range(count_attachments):
                file_name = msg.Attachments.Item(item + 1).Filename
                if 'USEmail' in file_name:
                    attach = msg.Attachments.Item(item+1)
                    attach.SaveAsFile(pasta_final+'\\'+file_name)

NameError: name 'message' is not defined

In [13]:
for subfolder in os.listdir(pasta_final):
    if subfolder.endswith('.zip'):
        file_name = pasta_final+'\\'+subfolder
        zip_ref = zipfile.ZipFile(file_name)
        zip_ref.extractall(file_name[:-4])
        zip_ref.close()
        os.remove(file_name)

In [15]:
path = pasta_final
base = 'Ajuste - Daily Stock Loan Fee Summary - '
try:
    for file in os.listdir(path):
        if len(os.listdir(path+'\\'+file))>2:
            for arquivo in os.listdir(path+'\\'+file):
                if ('.PDF' in arquivo)|('.pdf' in arquivo):
                    fileReader = PyPDF2.PdfFileReader(path+'\\'+file+'\\'+arquivo)
                    pageObj = fileReader.getPage(0)
                    for i in range(11,14):
                        data = pd.DataFrame(pageObj.extractText().split()).iloc[i][0]
                        if '/' in data:
                            data = str(pd.to_datetime(data[-14:-4], format = '%m/%d/%Y')).split()[0]               
                            for arquivo in os.listdir(path+'\\'+file):
                                if ('StockLoanFee' in arquivo)&(('.csv' in arquivo)|('.CSV' in arquivo)):
                                    os.rename(path+'\\'+file+'\\'+arquivo, path+'\\'+file+'\\'+base+str(data).split()[0]+'.csv')
except FileExistsError:
    print (file+'\\'+arquivo)

IndexError: single positional indexer is out-of-bounds

In [15]:
for subfolder in os.listdir(path):
    for item in os.listdir(path+'\\'+subfolder):
        if 'Ajuste' in item:
            try:
                os.rename(path+'\\'+subfolder+'\\'+item, path[:-8]+'\\'+item)
                print (item)
            except FileExistsError:
                continue
            else:
                break

In [ ]:
for subfolder in os.listdir(path):
    shutil.rmtree(path+'\\'+subfolder)